In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


## https://pypi.org/project/biobert-embedding/

In [ ]:
!pip3 install biobert-embedding

     |████████████████████████████████| 748.9MB 25kB/s 
     |████████████████████████████████| 133kB 58.4MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 7.4MB 45.9MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
  Created wheel for biobert-embedding: filename=biobert_embedding-0.1.2-cp37-none-any.whl size=5701 sha256=a710ec18655191f702d44f03b13f8c5dc146aeaac5d350eabe14f9fbffe496c1
  Stored in directory: /root/.cache/pip/wheels/ad/15/65/3fc6192a7cb7920672bb46d566173decb0875f35bbe03cd09d
  Created wheel for boto3: filename=boto3-1.17.43-py2.py3-none-any.whl size=128778 sha256=824c9983b813d35417e555ccf88433ff1183f49882c80dc52ba8d50d89faeb2a
  Stored in directory: /root/.cache/pip/wheels/dc/b4/95/0327754601f89dfdae855ca2280bdaf249e625afd0ef17a4a3
Successfully built biobert-embedding boto3
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.2.0 which is incompatible.
ERROR: torchtext 0.9.1 ha

In [ ]:
import nltk
nltk.download('punkt')  
nltk.download('stopwords')
import json
import pickle as pk

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
import pickle as pk
import os
from sklearn.cluster import KMeans
from sklearn import metrics
from collections import defaultdict
import json
from biobert_embedding.embedding import BiobertEmbedding

## Instantiate a BioBERT model

In [ ]:
biobert = BiobertEmbedding()

## Method below takes an icd_11 chapter and uses this chapter name to load the entities from the file system and then cluster

In [ ]:
def _generate_entity_clusters(icd_chapter, min_clusters = 20, max_clusters = 50):
  DATA_PATH = "medline_abstracts_w_entities"
  df = pk.load( open(os.path.join(DATA_PATH, f"{icd_chapter}.pk"), "rb"))

  lst_entities = []
  for dict_entities in list(df['Named_Entities_genia_w_BC5CDR'].values):
    try:
      lst_entities += list(dict_entities.values())[0]
    except (AttributeError, IndexError):
      continue
  lst_entities = list(set(lst_entities))

  embed_values = []
  track_entities = []
  for entity in lst_entities:
    if entity not in track_entities:
      track_entities.append(entity)

      entity_embedding = biobert.word_vector(entity)
      np_entity_embedding = [embed.numpy() for embed in entity_embedding]
      entity_embedding = np.mean(np.array(np_entity_embedding), axis=0)
      embed_values.append(entity_embedding)
      
  # Find the best value for k-number of clusters, iteratively
  max_silhouette_score = 0.0
  if max_clusters >= min_clusters:
    for k in range(min_clusters, max_clusters):
      if k % 5 == 0:
        print("Iteration ", k)
      kmeans = KMeans(n_clusters=k, random_state=0).fit(embed_values)
      labels = kmeans.labels_
      silhouette_avg = metrics.silhouette_score(embed_values, labels)
      if silhouette_avg >= max_silhouette_score:
        max_silhouette_score = silhouette_avg
        K_max = k
    print("Number of entities: ", len(embed_values))
    print("Maximum number of clusters:", K_max)
    # Run K-means with k that has given the best calinski score
    kmeans = KMeans(n_clusters=K_max, random_state=0).fit(embed_values)
    labels = kmeans.labels_

    # A dictionary of cluster label as a key and the entities that fall under that cluster
    dict_cluster_label_entities = defaultdict(list)
    for idx, label in enumerate(labels):
      dict_cluster_label_entities[label].append(lst_entities[idx])

    return dict_cluster_label_entities

## Call to the function above and write to a file system

In [ ]:
icd_chapter = "pregnancy_childbirth_and_the_puerperium_Medline"
dict_cluster_label_entities =_generate_entity_clusters(icd_chapter)


# Write the result to a file system
pk.dump(dict_cluster_label_entities, open(f"entity_clusters/{icd_chapter}.pk", "wb"))


## Read back one pickle file to confirm

In [ ]:
import os
import pickle as pk

DATA_PATH = "entity_clusters"
icd_chapter = "certain_infectious_or_parasitic_diseases_Medline"

dict_cluster_label_entities = pk.load( open(os.path.join(DATA_PATH, f"{icd_chapter}.pk"), "rb" ) )

In [ ]:
dict_cluster_label_entities

defaultdict(list,
            {0: ['Influenza A infection',
              'primary infection',
              'non-union',
              'maturation of cognitive, language and visual skills',
              'necrotizing pulmonary infection',
              'acute vision loss',
              'infectious complications',
              'gastrointestinal bleed',
              'Hepatitis C',
              'Respiratory infections',
              'Tuber infection',
              'immune-mediated disorders',
              'acute urinary retention',
              'Rd-R',
              'chronic infections',
              'Crohn disease',
              'nephrotic syndrome',
              'circulatory disturbance',
              'periodontal infections',
              'lower back ache',
              "Kaposi's sarcoma-associated herpesvirus",
              'CPA-1',
              'chronic illness',
              'streptococcal toxic shock syndrome',
              'postoperative cardiovascular and infec